2017 edition

# Building a music recommender system

As its name implies, a recommender system is a tool that helps predicting what a user may or may not like among a list of given items. In some sense, you can view this as an alternative to content search, as recommendation engines help users discover products or content that they may not come across otherwise. For example, Facebook suggests friends and pages to users. Youtube recommends videos which users may be interested in. Amazon suggests the products which users may need... Recommendation engines engage users to services, can be seen as a revenue optimization process, and in general help maintaining interest in a service.

In this notebook, we study how to build a simple recommender system: we focus on music recommendations, and we use a simple algorithm to predict which items users might like, that is called ALS, alternating least squares.

## Goals

In this lecture, we expect students to:

- Revisit (or learn) recommender algorithms

- Understand the idea of Matrix Factorization and the ALS algorithm (serial and parallel versions)

- Build a simple model for a real usecase: music recommender system

- Understand how to validate the results

## Steps

We assume students to work outside lab hours on the learning material. These are the steps by which we guide students, during labs, to build a good basis for the end-to-end development of a recommender system:

* Inspect the data using Spark SQL, and build some basic, but very valuable knowledge about the information we have at hand
* Formally define what is a sensible algorithm to achieve our goal: given the "history" of user taste for music, recommend new music to discover. Essentialy, we want to build a statistical model of user preferences such that we can use it to "predict" which additional music the user could like
* With our formal definition at hand, we will learn different ways to implement such an algorithm. Our goal here is to illustrate what are the difficulties to overcome when implementing a (parallel) algorithm
* Finally, we will focus on an existing implementation, available in the Apache Spark MLLib, which we will use out of the box to build a reliable statistical model

Now, you may think at this point we will be done!

Well, you'd better think twice: one important topic we will cover in all our Notebooks is **how to validate the results we obtain**, and **how to choose good parameters to train models** especially when using an "opaque" library for doing the job. As a consequence, we will focus on the statistical validation of our recommender system.

# Important note for grading
This notebook displays a series of questions, that we use to grade the work done. Since the following questions are rather basic, and do not change much from year to year, they only allow reaching the grade 10/20. Additional points can be gained by showing originality, depth, algorithmic design and implementations beyond that used in the notebook. Remember that this should become your own notebook: there is ample room for creativity!

# 1. Data

Understanding data is one of the most important part when designing any machine learning algorithm. In this notebook, we will use a data set published by Audioscrobbler - a music recommendation system for last.fm. Audioscrobbler is also one of the first internet streaming radio sites, founded in 2002. It provided an open API for “scrobbling”, or recording listeners’ plays of artists’ songs. last.fm used this information to build a powerful music recommender engine.

## 1.1. Data schema

Unlike a rating dataset which contains information about users' preference for products (one star, 3 stars, and so on), the datasets from Audioscrobbler only has information about events: specifically, it keeps track of how many times a user played songs of a given artist and the names of artists. That means it carries less information than a rating: in the literature, this is called explicit vs. implicit ratings.

### Reading material

- [Implicit Feedback for Inferring User Preference: A Bibliography](http://people.csail.mit.edu/teevan/work/publications/papers/sigir-forum03.pdf)
- [Comparing explicit and implicit feedback techniques for web retrieval: TREC-10 interactive track report](http://trec.nist.gov/pubs/trec10/papers/glasgow.pdf)
- [Probabilistic Models for Data Combination in Recommender Systems](http://mlg.eng.cam.ac.uk/pub/pdf/WilGha08.pdf)

The data we use in this Notebook is available in 3 files (these files are stored in our HDFS layer, in the directory  ```/datasets/lastfm```):

- **`user_artist_data.txt`**: It contains about 140,000+ unique users, and 1.6 million unique artists. About 24.2 million users’ plays of artists’ are recorded, along with their count. It has 3 columns separated by spaces: 

| UserID | ArtistID | PlayCount |
|----|----|----|
| ...|...|...|


- **`artist_data.txt`** : It prodives the names of each artist by their IDs. It has 2 columns separated by tab characters (`\t`).

| ArtistID | Name |
|---|---|
|...|...|

- **`artist_alias.txt`**: Note that when plays are scrobbled, the client application submits the name of the artist being played. This name could be misspelled or nonstandard. For example, "The Smiths", "Smiths, The", and "the smiths" may appear as distinct artist IDs in the data set, even though they are plainly the same. `artist_alias.txt` maps artist IDs that are known misspellings or variants to the canonical ID of that artist. The data in this file has 2 columns separated by tab characters (`\t`).

| MisspelledArtistID | StandardArtistID |
|---|---|
|...|...|



## 1.2. Understanding data: simple descriptive statistic

In order to choose or design a suitable algorithm for achieving our goals, given the data we have, we should first understand data characteristics. To start, we import the necessary packages to work with regular expressions, Data Frames, and other nice features of our programming environment.

In [19]:
import os
import sys
import re
import random
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from time import time

sqlContext = SQLContext(sc)
base = "/datasets/lastfm/"

In [20]:
from random import randint
from operator import add
rawUserArtistData = sc.textFile("train.csv")

def disambiguate(line):
    raw_list = line.split(',')
#     return (userID, finalArtistID, count)
    return ((raw_list[11], raw_list[2]), (float(raw_list[-1])*randint(1,4)) if (float(raw_list[-1]) == 1) else -1.0*randint(1,4))

header = rawUserArtistData.first()
rawUserArtistData = rawUserArtistData.filter(lambda x: x != header)
userArtistDataRDD = rawUserArtistData.map(lambda x: disambiguate(x))
#print(userArtistDataRDD.count())
userArtistDataRDD = userArtistDataRDD.reduceByKey(add)
#bArtistAlias.value
userArtistDataRDD.take(5)
#print(userArtistDataRDD.count())

[(('12682', '134212670'), -3.0),
 (('5896', '12792148'), 9.0),
 (('5248', '104631578'), -4.0),
 (('1921', '1101731'), -2.0),
 (('2758', '17170800'), -3.0)]

In [21]:
positiveUser = userArtistDataRDD.filter(lambda x: x[1] > 0)
nPositive = positiveUser.count()
print("Pos: ", nPositive)
negativeUser = userArtistDataRDD.filter(lambda x: x[1] < 0)
nNegative = negativeUser.count()
print("Neg: ", nNegative)

Pos:  2505932
Neg:  1403720


In [22]:
positiveSampleUser = positiveUser.sample(False, nNegative*1.0 / nPositive , seed=10)
nPositiveSample = positiveSampleUser.count()
print("Pos: ", nPositiveSample)

Pos:  1404144


In [38]:
trainData = positiveSampleUser.union(negativeUser)
nTrainData = trainData.count()
print("Train: ", nTrainData)

AttributeError: 'PipelinedRDD' object has no attribute 'unionAll'

In [24]:
userArtistDataRDD.take(5)

[(('12682', '134212670'), -3.0),
 (('581', '6026499'), 4.0),
 (('5248', '104631578'), -4.0),
 (('1921', '1101731'), -2.0),
 (('9331', '125089724'), -1.0)]

In [36]:
testData = sc.textFile("test.csv")

def disambiguate_test(line):
    raw_list = line.split(',')
#     return (userID, finalArtistID, count)
    return ((raw_list[12], raw_list[3]))

headerTest = testData.first()
testData = testData.filter(lambda x: x != headerTest)
testDataRDD = testData.map(lambda x: disambiguate_test(x))
#bArtistAlias.value
testDataRDD.take(5)

[('17698', '683078'),
 ('10525', '876497'),
 ('8716', '876497'),
 ('5443', '876500'),
 ('7600', '876504')]

In [26]:
def getMapVal(line):
    raw_list = line.split(',')
    return (raw_list[12], raw_list[0])

testIdMap = testData.map(lambda x: getMapVal(x)).collect()
print(testIdMap[0:5])

[('17698', '0'), ('10525', '1'), ('8716', '2'), ('5443', '3'), ('7600', '4')]


## 3.4 Training our statistical model
To train a model using ALS, we must use a preference matrix  as an input. MLLIB uses the class `Rating` to support the construction of a distributed preference matrix.

### Question 7

#### Question 7.1
Given RDD `userArtistDataRDD` in question 6.2, construct a new RDD `trainingData` by tranforming each item of it into a `Rating` object.

In [27]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [28]:

# allData = userArtistDataRDD.map(lambda r: Rating(r[0][0], r[0][1], r[1])).repartition(16).cache()
allData = trainData.map(lambda r: Rating(r[0][0], r[0][1], r[1])).repartition(16).cache()
allData.take(5)

[Rating(user=4529, product=3148226, rating=3.0),
 Rating(user=3348, product=3611377, rating=2.0),
 Rating(user=677, product=109703514, rating=1.0),
 Rating(user=3648, product=129436342, rating=2.0),
 Rating(user=7744, product=127614457, rating=8.0)]

#### Question 7.2
A model can be trained by using `ALS.trainImplicit(<training data>, <rank>)`, where:
* `training data` is the input data you decide to feed to the ALS algorithm
* `rank` is the number of laten features

We can also use some additional parameters to adjust the quality of the model. Currently, let's set 

- `rank=10`
- `iterations=5`
- `lambda_=0.01`
- `alpha=1.0` 

to build model.

In [29]:

t0 = time()
model = ALS.trainImplicit(allData, rank=200, iterations=20, lambda_=0.2, alpha=40.0)
t1 = time()
print("finish training model in %f secs" % (t1 - t0))


finish training model in 1474.034392 secs


In [30]:

! hdfs dfs -rm -R -f -skipTrash dsg17_2_model.spark
model.save(sc , 'dsg17_2_model.spark')


Deleted dsg17_2_model.spark


KeyboardInterrupt: 

In [ ]:

t0 = time()
model = MatrixFactorizationModel.load(sc, 'dsg17_2_model.spark')
t1 = time()
print("finish loading model in %f secs" % (t1 - t0))


# Predict

In [37]:

result = (model.predictAll(testDataRDD).collect())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 3062.0 failed 4 times, most recent failure: Lost task 7.3 in stage 3062.0 (TID 3611, spark-worker1-96813-prod, executor 16): java.lang.StackOverflowError
	at java.io.ObjectInputStream$BlockDataInputStream.readByte(ObjectInputStream.java:3069)
	at java.io.ObjectInputStream.readHandle(ObjectInputStream.java:1643)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1707)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:1986)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1535)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2231)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2155)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2013)
	at java.io.ObjectInputStream.readObject0(Objec

In [ ]:
scores = list(map(lambda x: x.rating, result))
print(np.sum(np.array(scores) <= 0))

In [ ]:
idDict = dict(testIdMap)
printResult = sorted(list(map(lambda x: (int(idDict[str(x.user)]), int(x.rating > 0)), result)), key=lambda x: x[0])
len(printResult)

In [ ]:
text_file = open("Output_spark_als.csv", "w")
text_file.write("sample_id,is_listened" + "\n") #sample_id,is_listened
x = 0
for i in range(19918):
    if (i == printResult[x][0]):
        text_file.write(str(i) + "," + str(printResult[x][1]) + "\n")
        x = x + 1
    else:
        text_file.write(str(i) + "," + str(randint(0,1)) + "\n")
text_file.close()

In [ ]:
################################ end of Implement
#############
###################
###################

## Extension

In [ ]:
from pyspark.mllib.clustering import GaussianMixture, GaussianMixtureModel

In [ ]:
data = sc.parallelize([np.array(x[1]) for x in model.userFeatures().collect()])
#data
gmm = GaussianMixture.train(data, 10)

In [ ]:
#print(gmm.gaussians)
#print(gmm.weights)
#predictedCluster = [gmm.predict(x) for x in data.collect()]
result = np.arange(148111)
i = 0
for x in data.collect():
    result[i] = gmm.predict(x)
    i = i + 1
len(result)

In [ ]:
for i in np.arange(10):
    print("Cluster %d has %d users" % ( i, len(result[result == i])))

COMMENTS: 


The clustering results shows that the user distribution is quite even between clusters.

Clustering the user will help us find which users have the same interest. Based on this result, we can make more diverse recommendation because we can treat them the same, when 1 user in a cluster listen to a new song that song can be propagated to all user in the same cluster. This benefit both the users and the artists and our company.



In [ ]:
prodMat = model.productFeatures().collect()
npProdMat = np.array([np.array(x[1]) for x in prodMat])
#clusterCenter = np.array(gmm.gaussians[8][0])

In [ ]:
print(set(x[1] for x in model.recommendProducts(120,10)))
clusterNum = gmm.predict([-0.013933692127466202, -0.19763502478599548, -0.24881640076637268, 0.2082066535949707, -0.04019957035779953, 0.20343655347824097, 0.10432276129722595, -0.1440502405166626, -0.06445351988077164, 0.12191807478666306])
clusterCenter = np.array(gmm.gaussians[7][0])
artistFeatureID = np.array([x[0] for x in prodMat])
#print(npProdMat.shape, " ", clusterCenter.shape)
resultScore = npProdMat.dot(clusterCenter)
resultIdx = sorted(range(len(resultScore)), key=lambda k: resultScore[k])
artistSortedId = [artistFeatureID[i] for i in resultIdx[0:10]]
print(artistSortedId)
#print(resultScore)
#npProdMat.shape

Comments:


This is the comparison between cluster center based recommendation and user recommendation. The recommendation list is different from the user based recommendation. It proves that we can make diverse recommendation to user based on users with the same music taste.

## 3.5 Evaluating Recommendation Quality 

In this section, we study how to evaluate the quality of our model. It's hard to say how good the recommendations are.
One of serveral methods approach to evaluate  a recommender based on its ability to rank good items (artists) high in a list of recommendations. The problem is how to define "good artists". Currently, by training all data, "good artists" is defined as "artists the user has listened to", and the recommender system has already received all of this information as input. It could trivially return the users previously-listened artists as top recommendations and score perfectly. Indeed, this is not useful, because the recommender's is used to recommend artists that the user has **never** listened to. 

To overcome that problem, we can hide the some of the artist play data and only use the rest to train model. Then, this held-out data can be interpreted as a collection of "good" recommendations for each user. The recommender is asked to rank all items in the model, and the rank of the held-out artists are examined. Ideally the recommender places all of them at or near the top of the list.

The recommender's score can then be computed by comparing all held-out artists' ranks to the rest.  The fraction of pairs where the held-out artist is ranked higher is its score. 1.0 is perfect, 0.0 is the worst possible score, and 0.5 is the expected value achieved from randomly ranking artists. 

AUC(Area Under the Curve) can be used as a metric to evaluate model. It is also viewed as the probability that a randomly-chosen "good" artist ranks above a randomly-chosen "bad" artist.

Next, we split the training data into 2 parts: `trainData` and `cvData` with ratio 0.9:0.1 respectively, where `trainData` is the dataset that will be used to train model. Then we write a function to calculate AUC to evaluate the quality of our model.


### Question 9

#### Question 9.1
Split the data into `trainData` and `cvData` with ratio 0.9:0.1 and use the first part to train a statistic model with:

- rank=10
- iterations=5
- lambda_=0.01
- alpha=1.0

In [ ]:

trainData, cvData = allData.randomSplit([0.9, 0.1])
trainData.cache()
cvData.cache()


In [ ]:

t0 = time()
model = ALS.trainImplicit(trainData, rank = 10, iterations=5, lambda_=0.01, alpha=1.0)
t1 = time()
print("finish training model in %f secs" % (t1 - t0))


#### Area under the ROC curve: a function to compute it

In [ ]:
# Get all unique artistId, and broadcast them
allItemIDs = np.array(allData.map(lambda x: x[1]).distinct().collect())
bAllItemIDs = sc.broadcast(allItemIDs)

In [ ]:
from random import randint

# Depend on the number of item in userIDAndPosItemIDs,
# create a set of "negative" products for each user. These are randomly chosen
# from among all of the other items, excluding those that are "positive" for the user.
# NOTE 1: mapPartitions operates on many (user,positive-items) pairs at once
# NOTE 2: flatMap breaks the collections above down into one big set of tuples
def xtractNegative(userIDAndPosItemIDs):
    def pickEnoughNegatives(line):
        userID = line[0]
        posItemIDSet = set(line[1])
        #posItemIDSet = line[1]
        negative = []
        allItemIDs = bAllItemIDs.value
        # Keep about as many negative examples per user as positive. Duplicates are OK.
        i = 0
        while (i < len(allItemIDs) and len(negative) < len(posItemIDSet)):
            itemID = allItemIDs[randint(0,len(allItemIDs)-1)]
            if itemID not in posItemIDSet:
                negative.append(itemID)
            i += 1
        
        # Result is a collection of (user,negative-item) tuples
        return map(lambda itemID: (userID, itemID), negative)

    # Init an RNG and the item IDs set once for partition
    # allItemIDs = bAllItemIDs.value
    return map(pickEnoughNegatives, userIDAndPosItemIDs)

def ratioOfCorrectRanks(positiveRatings, negativeRatings):
    
    # find number elements in arr that has index >= start and has value smaller than x
    # arr is a sorted array
    def findNumElementsSmallerThan(arr, x, start=0):
        left = start
        right = len(arr) -1
        # if x is bigger than the biggest element in arr
        if start > right or x > arr[right]:
            return right + 1
        mid = -1
        while left <= right:
            mid = (left + right) // 2
            if arr[mid] < x:
                left = mid + 1
            elif arr[mid] > x:
                right = mid - 1
            else:
                while mid-1 >= start and arr[mid-1] == x:
                    mid -= 1
                return mid
        return mid if arr[mid] > x else mid + 1
    
    ## AUC may be viewed as the probability that a random positive item scores
    ## higher than a random negative one. Here the proportion of all positive-negative
    ## pairs that are correctly ranked is computed. The result is equal to the AUC metric.
    correct = 0 ## L
    total = 0 ## L
    
    # sorting positiveRatings array needs more cost
    #positiveRatings = np.array(map(lambda x: x.rating, positiveRatings))

    negativeRatings = list(map(lambda x:x.rating, negativeRatings))
    
    #np.sort(positiveRatings)
    negativeRatings.sort()# = np.sort(negativeRatings)
    total = len(positiveRatings)*len(negativeRatings)
    
    for positive in positiveRatings:
        # Count the correctly-ranked pairs
        correct += findNumElementsSmallerThan(negativeRatings, positive.rating)
        
    ## Return AUC: fraction of pairs ranked correctly
    return float(correct) / total

def calculateAUC(positiveData, bAllItemIDs, predictFunction):
    # Take held-out data as the "positive", and map to tuples
    positiveUserProducts = positiveData.map(lambda r: (r[0], r[1]))
    # Make predictions for each of them, including a numeric score, and gather by user
    positivePredictions = predictFunction(positiveUserProducts).groupBy(lambda r: r.user)
    
    # Create a set of "negative" products for each user. These are randomly chosen 
    # from among all of the other items, excluding those that are "positive" for the user. 
    negativeUserProducts = positiveUserProducts.groupByKey().mapPartitions(xtractNegative).flatMap(lambda x: x)
    # Make predictions on the rest
    negativePredictions = predictFunction(negativeUserProducts).groupBy(lambda r: r.user)
    
    return (
            positivePredictions.join(negativePredictions)
                .values()
                .map(
                    lambda positive_negativeRatings: ratioOfCorrectRanks(positive_negativeRatings[0], positive_negativeRatings[1])
                )
                .mean()
            )

####  Question 9.2
Using part `cvData` and function `calculateAUC` to compute the AUC of the trained model.

In [ ]:

t0 = time()
auc = calculateAUC(cvData, bAllItemIDs, model.predictAll)
t1 = time()
print("auc=",auc)
print("finish in %f seconds" % (t1 - t0))


the AUC is very high for this data

#### Question 9.3
Now we have the UAC of our model, it’s helpful to benchmark this against a simpler approach. For example, consider recommending the globally most-played artists to every user. This is not personalized, but is simple and may be effective. 

Implement this simple pupolarity-based prediction algorithm, evaluate its AUC score, and compare to the results achieved by the more sophisticated ALS algorithm.

In [ ]:
from operator import add

bListenCount = sc.broadcast(trainData.map(lambda r: (r[1], r[2])).reduceByKey(add).collectAsMap())
def predictMostListened(allData):
    return allData.map(lambda r: Rating(r[0], r[1], bListenCount.value.get(r[1], 0.0)))


In [ ]:

auc = calculateAUC(cvData, bAllItemIDs, predictMostListened)
print(auc)


Comments:

The simple method achieve lower accuracy than the ALS method but the different is quite insignificant.

We should use different metric to evaluate this algorithm to ensure the accuracy is correct. AUC is just for reference it should not be used to evaluate in this situation. There may be artist that are listened to once out of curiosity. Our there may be artists that are not yet listened to but is a positive.

## 3.6 Personalized recommendations with ALS

In the previous section, we build our models with some given paramters without any knowledge about them. Actually, choosing the best parameters' values is very important. It can significantly affect the quality of models. Especially, with the current implementation of ALS in MLLIB, these parameters are not learned by the algorithm, and must be chosen by the caller. The following parameters should get consideration before training models:

* `rank = 10`: the number of latent factors in the model, or equivalently, the number of columns $k$ in the user-feature and product-feature matrices. In non-trivial cases, this is also their rank. 

* `iterations = 5`: the number of iterations that the factorization runs. Instead of runing the algorithm until RMSE converged which actually takes very long time to finish with large datasets, we only let it run in a given number of iterations. More iterations take more time but may produce a better factorization.

* `lambda_ = 0.01`: a standard overfitting parameter. Higher values resist overfitting, but values that are too high hurt the factorization's accuracy.

*  `alpha = 1.0`: controls the relative weight of observed versus unobserved userproduct interactions in the factorization. 

Although all of them have impact on the models' quality, `iterations` is more of a constraint on resources used in the factorization. So, `rank`, `lambda_` and `alpha` can be considered hyperparameters to the model. 
We will try to find "good" values for them. Indeed, the values of hyperparameter are not necessarily optimal. Choosing good hyperparameter values is a common problem in machine learning. The most basic way to choose values is to simply try combinations of values and evaluate a metric for each of them, and choose the combination that produces the best value of the metric. 


### Question 10

#### Question 10.1
For simplicity, assume that we want to explore the following parameter space: $ rank \in \{10, 50\}$, $lambda\_ \in \{1.0, 0.0001\}$ and $alpha \in \{1.0, 40.0\}$.

Find the best combination of them in terms of the highest AUC value.

In [ ]:

evaluations = []

for rank in [10, 50]:
    for lambda_ in [1.0, 0.0001]:
        for alpha in [1.0, 40.0]:
            print("Train model with rank=%d lambda_=%f alpha=%f" % (rank, lambda_, alpha))
            # with each combination of params, we should run multiple times and get avg
            # for simple, we only run one time.
            model = ALS.trainImplicit(trainData, iterations=5, rank=rank, lambda_=lambda_, alpha=alpha)
            
            auc = calculateAUC(cvData, bAllItemIDs, model.predictAll)
            
            evaluations.append(((rank, lambda_, alpha), auc))
            
            unpersist(model)




In [ ]:
evaluations.sort(key=lambda x: x[1], reverse=True)
                 
evalDataFrame = pd.DataFrame(data=evaluations)
#print(evalDataFrame)

trainData.unpersist()
cvData.unpersist()
evaluations

Comments:

The parameters with the highest AUC is: rank = 50, lambda = 1.0, and alpha=40.0
Rank 50 is more accurate than rank 10 means that more features will contribute to the accuracy of the recommendation. However, we should examine the rank closely so as not to overfit the training data with too many features.

Lambda is the regularization parameter is 1 meaning that stronger regularization is needed in this case.

#### Question 10.2 

Using "optimal" hyper-parameters in question 10.1, re-train the model and show top-5 artist names recommendated for user 2093760.

In [ ]:

model = ALS.trainImplicit(trainData, rank=50, lambda_=1.0, alpha=40.0)
allData.unpersist()

userID = 2093760
recommendations = model.recommendProducts(userID, 5)

recommendedProductIDs = set(x[1] for x in recommendations)

recList = artistByID.filter(lambda x: artistNames(x)).values().collect()
print(recList)

unpersist(model)
# '50 Cent', 'Snoop Dogg', 'Usher', 'Nas', 'Jay-Z', 'Kanye West',

Comments:

The result for user 2093760 consist with the most listened to artist which means this user is an average user not an outlier.

Compared to the original parameter in the previous question, with the optimal parameter the ranking of recommended artist is a little different, however, they are all famous rap artists. This validate our hypothesis that we can make cheaper and more robust recommendation by using clustering of user or artist.


# Summary

In this notebook, we introduce an algorithm to do matrix factorization and the way of using it to make recommendation. Further more, we studied how to build a large-scale recommender system on SPARK using ALS algorithm and evaluate its quality. Finally, a simple approach to choose good parameters is mentioned.

# References

- The example in section 2 is taken from [Recommender system](infolab.stanford.edu/~ullman/mmds/ch9.pdf)